### 잠재요인 협업 필터링
* 사용자-아이템 평점 매트릭스속에 숨어 있는 잠재요인 추출해 추천 예측
* 대규모 다차원 행렬 SVD(6장)과 같은 차원축소 기법, 행렬분해
* 사용자 아이템 평점 점수를 아래와 같이 장르를 매개로 분해
    + R(M X N) = P(M X K) * (K X N)
    + P 사용자별 장르선호도, Q 영화별 장르요소 => 장르를 중간 매개요소
    + R = P * Q.T  ==> 
* 행렬분해는 SVD사용, but 널(NaN)값이 없는 행렬에만 적용
    + R 사용자-아이템 행렬에는 널값이 많음
    + SGD(Stochastic Graident Descent, 확률적 경사하강법)이나 ALS(Alternating Least Squares)방식이용 SVD수행

#### SGD(Stochastic Graident Descent, 확률적 경사하강법)
* P*Q.T에 의해 예측된 R 값이 실제 R값과 가장 최소의 오류를 가지도록 반복적 비용함수 최적화
* P와 Q의 행렬을 비용이 최소화되도록 각각 업데이트 2 -3번 작업
* L2규제 고려, 학습률 고려, 회귀계수를 반복적으로 적용

In [1]:
import numpy as np
# 원본 행렬 R생성, 분해행렬 P와 Q초기화, 잠재요인차원 K는 3으로 설정
R = np.array( [[4, np.NaN, np.NaN, 2, np.NaN],    
               [np.NaN, 5, np.NaN, 3, 1],    
               [np.NaN, np.NaN, 3, 4, 4],    
               [5, 2, 1, 2, np.NaN]    
                ])
num_users, num_items = R.shape
K=3

# P와 Q 행령의 크기를 지정하고 정규분포를 가진 임의의  값으로 입력합니다.
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

#### 실제 R행렬과 예측행렬의 오차를 구하는 get_rmse()
* 실제 R행렬의 널이 아닌 값의 위치 인덱스를 추출해 
* 이 인덱스에 있는 실제 R행렬갑과 분해된 P,Q를 이용해 다시 조합된 예측행렬값의 RMSE값 반환

In [2]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse


#### SGD기반 행렬분해 수행
* 먼저 R에서 널 값을 제외한 데이터의 행렬인덱스 추출
* steps=100 반복해서 업데이트할 횟수
* learning_rate 학습률 = n
* r_lambda L2규제 계수 람다
* p' = p + n(e*q- 람다*p)     q' = q + n(e*q - 람다*q) 업데이트      e 오류 = r - r^ 


In [3]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장. 
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=1000
learning_rate=0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트. 
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)



### iteration step :  0  rmse :  3.2388050277987723
### iteration step :  50  rmse :  0.4876723101369648
### iteration step :  100  rmse :  0.1564340384819247
### iteration step :  150  rmse :  0.07455141311978046
### iteration step :  200  rmse :  0.04325226798579314
### iteration step :  250  rmse :  0.029248328780878973
### iteration step :  300  rmse :  0.022621116143829466
### iteration step :  350  rmse :  0.019493636196525135
### iteration step :  400  rmse :  0.018022719092132704
### iteration step :  450  rmse :  0.01731968595344266
### iteration step :  500  rmse :  0.016973657887570753
### iteration step :  550  rmse :  0.016796804595895633
### iteration step :  600  rmse :  0.01670132290188466
### iteration step :  650  rmse :  0.01664473691247669
### iteration step :  700  rmse :  0.016605910068210026
### iteration step :  750  rmse :  0.016574200475705
### iteration step :  800  rmse :  0.01654431582921597
### iteration step :  850  rmse :  0.01651375177473524
### iterati

#### 분해된 P와 Q의 함수를 P*Q.T 로 예측행렬
* 널인 값이 예측값으로 채워짐

In [4]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]
